# Grid extrapolation of low-carbon generation

## Imports

In [1]:
# Add parent dir to the path, so we can load the module

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))


In [2]:
import pandas as pd

import energy_insights.params_library.basic_source
from energy_insights.region import *
from energy_insights.country_grid import CountryGrid
from energy_insights.country_grid_stats import CountryGridStats
from energy_insights.hourly_data_extrapolator import HourlyDataExtrapolator
from energy_insights.yearly_grid_plot import YearlyGridPlot
from energy_insights.yearly_filter import YearlyFilter
from energy_insights.params_library.interconnectors import get_interconnectors
from energy_insights.params_utils import merge_config_into_scenario
from energy_insights.plot_strings import get_grid_strings
from energy_insights.plot_utils import get_scenario_out_dir, remove_scenario_out_dir_if_empty, get_analysis_out_dir
from energy_insights.sources.basic_source import get_basic_sources


## Config

In [3]:
runs = [
    {
        'config': {
            "year": 2021,
            "analysis_name": "grid-plot",
            "filter": {
                'week_sampling': 4
            },
            "output": {
                'regions': 'aggregate',
                'separate_excess': False,
                'format': "png",
                'dpi': 100,
                'parts': ['titles', 'weeks', 'week_summary', 'year_stats'],
            }
        },
        'scenarios': [
            # {  # TODO: Clean up / make work after refactoring. This is tricky w.r.t. "current".
            #     "name": "2021-soucasnost",
            #     "countries": {
            #         CZECHIA: {"load_factors": "current"},
            #         GERMANY: {"load_factors": "current"},
            #     },
            #     "interconnectors": "2021",
            # },
            # {
            #     "name": "2030-plan",
            #     "countries": {
            #         CZECHIA: {
            #             "load_factors": "cz-2030-basic",
            #             "basic_sources": "cz-2030-basic",
            #         },
            #         GERMANY: {
            #             "load_factors": "de-2030-government-plans-achieved",
            #             "basic_sources": "de-2030-government-plans-achieved",
            #         }
            #     },
            #     "interconnectors": "2021",
            # },
            # {
            #     "name": "2030-plan-with-cz",
            #     "countries": {
            #         CZECHIA: {
            #             "load_factors": "cz-2050-explainer-scenario",
            #             "basic_sources": "cz-2050-explainer-biomass-scenario",
            #         },
            #         GERMANY: {
            #             "load_factors": "de-2030-government-plans-achieved",
            #             "basic_sources": "de-2030-government-plans-achieved",
            #         },
            #     },
            #     "interconnectors": "2021",
            # },
        ]
    },
]


## Code

In [4]:
extrapolator = HourlyDataExtrapolator(data_path='../data')

for run_number, run in enumerate(runs):
    config = run['config']
    print(f"\nRun number #{run_number}: {config['analysis_name']}")

    all_stats_per_scenario = []

    for scenario in run['scenarios']:
        print(f"\nRunning scenario ‘{scenario['name']}’")

        params = merge_config_into_scenario(config, scenario)
        # Expand params from a library
        data_year = params['year']
        interconnectors = get_interconnectors(
            params.get('interconnectors', {}), params["countries"].keys())

        grids: dict[Region, CountryGrid] = {}
        load_factors_separate: dict[Region, dict] = {}
        installed_separate: dict[Region, dict] = {}
        for country, country_params in params["countries"].items():
            load_factors = country_params["load_factors"]
            load_factors_separate[country] = load_factors
            installed_map_gw = {
                key: source.capacity_mw for key, source in basic_sources.items()
            }

            basic_sources = get_basic_sources(country_params["basic_sources"])
            data = extrapolator.extrapolate_hourly_country_data(
                country, data_year, None, data_year, load_factors, basic_sources, installed_map_gw, {},
                load_hydro_from_pecd=False, load_demand_from_pecd=False)
            installed_separate[country] = installed_map_gw

            grids[country] = CountryGrid(country, data, basic_sources, [], [])

        extrapolator.estimate_interconnector_flows(grids, interconnectors)
        title_format, subtitle_format, factors_str, name = get_grid_strings(
            params, load_factors_separate, installed_separate)
        out_dir = get_scenario_out_dir(params["name"], params.get("analysis_name", None))

        yearly_filter = YearlyFilter.build(params["filter"])
        output = params["output"]
        only_aggregate: bool = output.get('regions', 'aggregate') == 'aggregate'
        grids_to_plot = CountryGrid.aggregate_grids(
            CountryGrid.filter_grids(grids, yearly_filter), only_aggregate)

        stats_to_plot: dict[Region, CountryGridStats] = {}
        for region, grid in grids_to_plot.items():
            stat = CountryGridStats(region, grid, interconnectors, params["name"])
            stats_to_plot[region] = stat
            all_stats_per_scenario += stat.get_stats_for_logging()

        plot = YearlyGridPlot(stats_to_plot, data_year, yearly_filter, output,
                              title_format, subtitle_format, factors_str, out_dir, name)
        plot.print_graph()
        remove_scenario_out_dir_if_empty(out_dir)

    if all_stats_per_scenario:
        analysis_df = pd.DataFrame(all_stats_per_scenario)
        out_dir = get_analysis_out_dir(config.get("analysis_name", None))
        analysis_df.to_csv(out_dir / "analysis.csv")
        analysis_df.pivot(index=['name', 'region', 'season', 'stat'],
                          columns='source', values='val').to_csv(out_dir / "analysis-pivot.csv")



Run number #0: grid-plot

Run number #1: topaz-plot

Running scenario ‘cz-2030-topaz-basic’
Plotting yearly graph for CZ
Plotting yearly graph... Done

Running scenario ‘cz-2030-topaz-advanced’
Plotting yearly graph for CZ
Plotting yearly graph... Done


<Figure size 1339.2x1728 with 0 Axes>